# Testing Agents v2 in AI Foundry
This notebook creates and runs an agent from AI Foundry

Best way to set it up is with [uv](https://docs.astral.sh/uv/)

1. in `agents` directory run `uv sync` (if python is missing, install it using `uv python install`)
2. in VSCode press [Ctrl+Shift+P] and select `Python: Select Interpreter`, choose the one from `agents` directory: `./agents/.venv/bin/python3.xx`

## Step 2
Create Project client

In [ ]:
import os
from azure.identity import DefaultAzureCredential, AzureDeveloperCliCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(override=True)

endpoint = os.environ.get("AZURE_AI_FOUNDRY_CONNECTION_STRING")
deployment_name = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", None)
tenant_id = os.environ.get("AZURE_TENANT_ID", None)

ai_agent_settings = {
    "endpoint": endpoint,
    "model_deployment_name": deployment_name,
    "api_version": api_version,
}
print(ai_agent_settings)

if os.environ.get("USE_AZURE_DEV_CLI") == "true":
    print("Using Azure Developer CLI Credential")

creds = (
    AzureDeveloperCliCredential(tenant_id=tenant_id)
    if os.environ.get("USE_AZURE_DEV_CLI") == "true"
    else DefaultAzureCredential()
)
# uncomment to test token aquisition
# token_test  = creds.get_token("https://ai.azure.com")
# print(f"Token for https://ai.azure.com: {token_test.token[:10]}...")

client = AIProjectClient(endpoint=endpoint, credential=creds)

# List agents
print("\n --- Agents ---")
agents = client.agents.list()
all_agents = []
for agent in agents:
    all_agents.append(agent)
    print(
        f"Agent ID: {agent.id}, Name: {agent.name}, version: {agent.versions.latest.version} Properties: {agent.as_dict()}"
    )

# List connections
print("\n --- Connections ---")
connections = client.connections.list()
model_gateway_connection = None
for connection in connections:
    print(
        f"Connection ID: {connection.id}, Name: {connection.name}, Type: {connection.type} Default: {connection.is_default}"
    )
    if connection.type == "ModelGateway":
        model_gateway_connection = connection.name
        print(f"  - Model gateway connection found: {model_gateway_connection}")

## Create Agent

In [ ]:
# check if agent "MyV2Agent" exists
agent_names = [agent.name for agent in all_agents]
if "MyV2Agent" not in agent_names:
    agent = client.agents.create(
        name="MyV2Agent",
        definition=PromptAgentDefinition(
            model=f'{model_gateway_connection}/{deployment_name}',
            instructions="You are a helpful assistant that answers general questions",
        ),
    )
    print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.versions.latest.version})")
else:
    agent = client.agents.update(agent_name="MyV2Agent", definition=PromptAgentDefinition(
                model=f'{model_gateway_connection}/{deployment_name}',
                instructions="You are a helpful assistant that answers general questions",
            ) )
    print(f"Agent updated (id: {agent.id}, name: {agent.name}, version: {agent.versions.latest.version})")

## Run Agent

In [ ]:
openai_client = client.get_openai_client()
conversation = openai_client.conversations.create(
            items=[{"type": "message", "role": "user", "content": "What is the size of France in square miles?"}],
        )
print(f"Created conversation with initial user message (id: {conversation.id})")

response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    input="",
)
print(f"Response output: {response.output_text}")